In [0]:
# 03 - Fraud Monitoring & Alert System
# **Purpose**: Monitor fraud patterns, generate alerts, and create compliance reports
# **Schedule**: Run daily after Gold layer processing completes

from pyspark.sql.functions import col, current_timestamp, split, when, size, expr, current_date
from pyspark.sql.types import StructType, StructField, TimestampType, IntegerType, BooleanType

# Read from Gold tables
gold_claims = spark.table("medisure_jen.gold.gold_claims_analytics")
fraud_alerts = spark.table("medisure_jen.gold.gold_realtime_fraud_alerts")
provider_performance = spark.table("medisure_jen.gold.gold_provider_performance")

# Display current day's fraud alerts
display(spark.sql("""
SELECT * FROM medisure_jen.gold.gold_realtime_fraud_alerts 
ORDER BY alert_severity DESC, alert_timestamp DESC
"""))

# 1.2 Fraud Trends Analysis
# Analyze fraud trends over time
fraud_trends = spark.sql("""
SELECT 
  processing_month,
  COUNT(*) as total_claims,
  SUM(CASE WHEN fraud_risk_score > 0.7 THEN 1 ELSE 0 END) as high_risk_claims,
  ROUND(AVG(fraud_risk_score), 3) as avg_fraud_score,
  ROUND(SUM(CASE WHEN fraud_risk_score > 0.7 THEN claim_amount ELSE 0 END), 2) as high_risk_amount
FROM medisure_jen.gold.gold_claims_analytics
WHERE processing_month >= date_format(add_months(current_date(), -6), 'yyyy-MM')
GROUP BY processing_month
ORDER BY processing_month
""")

display(fraud_trends)

# 2. Compliance Reporting
# 2.1 Provider Compliance Report

provider_compliance_report = spark.sql("""
SELECT 
  provider_id,
  provider_name,
  tin,
  total_claims,
  total_amount,
  avg_claim_amount,
  avg_fraud_score,
  high_risk_claims,
  ROUND((high_risk_claims / total_claims) * 100, 2) as high_risk_percentage,
  CASE 
    WHEN (high_risk_claims / total_claims) > 0.3 THEN 'REVIEW REQUIRED'
    WHEN (high_risk_claims / total_claims) > 0.1 THEN 'MONITOR'
    ELSE 'COMPLIANT'
  END as compliance_status
FROM medisure_jen.gold.gold_provider_performance
WHERE reporting_period = date_format(current_date(), 'yyyy-MM')
ORDER BY high_risk_percentage DESC
""")

display(provider_compliance_report)

# 2.2 Member Risk Profiling

member_risk_profiles = spark.sql("""
SELECT 
  member_id,
  first_name,
  last_name,
  claims_count,
  total_claimed,
  member_risk_score,
  CASE 
    WHEN member_risk_score > 0.8 THEN 'HIGH RISK'
    WHEN member_risk_score > 0.5 THEN 'MEDIUM RISK'
    ELSE 'LOW RISK'
  END as risk_category
FROM medisure_jen.gold.gold_member_claims_summary
WHERE summary_period = date_format(current_date(), 'yyyy-MM-dd')
ORDER BY member_risk_score DESC
LIMIT 100
""")

display(member_risk_profiles)

# 3. Alert Generation & Notification
# 3.1 Critical Alerts Detection - FIXED WITH NAME SPLITTING
# Find critical alerts that need immediate attention
critical_alerts_query = """
SELECT 
  a.claim_id,
  a.member_id,
  a.provider_id,
  a.claim_amount,
  a.diagnosis_code,
  a.alert_severity,
  a.alert_reason,
  a.alert_timestamp,
  split(m.member_name, ' ')[0] as first_name,
  array_join(slice(split(m.member_name, ' '), 2, size(split(m.member_name, ' ')) - 1), ' ') as last_name,
  p.provider_name
FROM medisure_jen.gold.gold_realtime_fraud_alerts a
LEFT JOIN medisure_jen.silver.silver_members m ON a.member_id = m.member_id
LEFT JOIN medisure_jen.silver.silver_providers p ON a.provider_id = p.provider_id
WHERE a.alert_severity = 'Critical'
  AND date(a.alert_timestamp) = current_date()
ORDER BY a.claim_amount DESC
"""

critical_alerts = spark.sql(critical_alerts_query)

# Alternative approach using DataFrame API if SQL doesn't work
if critical_alerts.count() == 0:
    print("Trying alternative approach with DataFrame API...")
    alerts_df = spark.table("medisure_jen.gold.gold_realtime_fraud_alerts")
    members_df = spark.table("medisure_jen.silver.silver_members")
    providers_df = spark.table("medisure_jen.silver.silver_providers")
    
    critical_alerts = (alerts_df
        .filter((col("alert_severity") == "Critical") & (col("alert_timestamp").cast("date") == current_date()))
        .join(members_df, "member_id", "left")
        .join(providers_df, "provider_id", "left")
        .withColumn("name_parts", split(col("member_name"), " "))
        .withColumn("first_name", col("name_parts")[0])
        .withColumn("last_name", 
                   when(size(col("name_parts")) > 1, 
                        expr("array_join(slice(name_parts, 2, size(name_parts) - 1), ' ')"))
                   .otherwise(""))
        .select(
            "claim_id", "member_id", "provider_id", "claim_amount", 
            "diagnosis_code", "alert_severity", "alert_reason", "alert_timestamp",
            "first_name", "last_name", "provider_name"
        )
        .orderBy(col("claim_amount").desc())
    )

display(critical_alerts)

# 3.2 Email Alert Function
def send_fraud_alert_email(alert_data):
    """
    Send email alert for critical fraud cases
    """
    critical_count = alert_data.count()
    
    if critical_count > 0:
        # Prepare email content
        subject = f"🚨 MediSure Fraud Alert: {critical_count} Critical Cases Detected"
        
        message = f"""
        Critical Fraud Alerts - {date.today()}
        Number of critical alerts: {critical_count}
        
        Top 5 Critical Cases:
        
        """
        
        # Add top 5 cases to email
        for row in alert_data.limit(5).collect():
            # Handle potential None values
            first_name = row.first_name if row.first_name else "Unknown"
            last_name = row.last_name if row.last_name else ""
            provider_name = row.provider_name if row.provider_name else "Unknown Provider"
            claim_amount = float(row.claim_amount) if row.claim_amount else 0.0
            alert_reason = row.alert_reason if row.alert_reason else "Unknown"
            
            message += f"""
            
            """
        
        message += "
        
            Claim ID
            Member
            Provider
            Amount
            Reason
        
                {row.claim_id}
                {first_name} {last_name}
                {provider_name}
                ${claim_amount:,.2f}
                {alert_reason}
            "
        
        # Send email (configure your email settings)
        print(f"Would send email with subject: {subject}")
        print(f"To: compliance-team@medisure.com")
        print(f"Body: {message}")
        
        # configure email here
        # dbutils.notify.send(
        #     subject=subject,
        #     body=message,
        #     recipients=["compliance-team@medisure.com"]
        # )
        
        return True
    else:
        print("No critical alerts to report today.")
        return False

# Send email alerts
email_sent = send_fraud_alert_email(critical_alerts)

# 3.3 Save Compliance Reports

# Save reports for auditing
(provider_compliance_report
 .write
 .format("delta")
 .mode("overwrite")
 .option("overwriteSchema", "true")
 .saveAsTable("medisure_jen.audit.provider_compliance_daily"))

(member_risk_profiles
 .write
 .format("delta")
 .mode("overwrite")
 .option("overwriteSchema", "true")
 .saveAsTable("medisure_jen.audit.member_risk_daily"))

# 4. Operational Monitoring
# 4.1 Pipeline Health Check
# Check data quality metrics
data_quality_check = spark.sql("""
SELECT 
  current_timestamp() as check_timestamp,
  (SELECT COUNT(*) FROM medisure_jen.gold.gold_claims_analytics) as total_claims,
  (SELECT COUNT(*) FROM medisure_jen.gold.gold_realtime_fraud_alerts 
   WHERE date(alert_timestamp) = current_date()) as today_alerts,
  (SELECT ROUND(AVG(fraud_risk_score), 3) 
   FROM medisure_jen.gold.gold_claims_analytics) as overall_fraud_score
""")

display(data_quality_check)

# 4.2 Log Monitoring Results
# Log monitoring results
schema = StructType([
    StructField("check_timestamp", TimestampType(), False),
    StructField("critical_alerts", IntegerType(), False),
    StructField("providers_needing_review", IntegerType(), False),
    StructField("email_sent", BooleanType(), False)
])

monitoring_log = spark.createDataFrame([(
    current_timestamp(),
    critical_alerts.count(),
    provider_compliance_report.filter(col("compliance_status") != "COMPLIANT").count(),
    email_sent
)], schema)

(monitoring_log
 .write
 .format("delta")
 .mode("append")
 .option("overwriteSchema", "true")
 .saveAsTable("medisure_jen.audit.fraud_monitoring_log"))

# 5. Summary Output
# Print summary
print("="*60)
print("FRAUD MONITORING SUMMARY")
print("="*60)
print(f"Execution Time: {datetime.now()}")
print(f"Critical Alerts Found: {critical_alerts.count()}")
print(f"Providers Needing Review: {provider_compliance_report.filter(col('compliance_status') != 'COMPLIANT').count()}")
print(f"Email Alert Sent: {'Yes' if email_sent else 'No'}")
print("="*60)